In [26]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Load the dataset
file_path = '/Users/fionamagee/Desktop/Sprint2/dataset1.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
columns_to_drop = ['ObjectId', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor']
data_cleaned = data.drop(columns=columns_to_drop)

# Rename columns
data_cleaned.rename(columns={'Country': 'country'}, inplace=True)

# Convert the dataset from wide format to long format
data_tidy = pd.melt(data_cleaned, id_vars=['country'], var_name='year', value_name='temperature_change')

# Remove 'F' from year
data_tidy['year'] = data_tidy['year'].str.replace('F', '')

# Create the Dash app
app = Dash(__name__)
#server=app.server
app.layout = html.Div([
    # Title centered at the top of the screen
    html.Div([
        html.H1('The Impacts of Climate Change', style={'textAlign': 'center'}),
    ], style={'width': '100%', 'textAlign': 'center'}),

    # Main content area with graph, text boxes, and controls
    html.Div([
        # Graph, its associated text box, and controls
        html.Div([
            # Controls (Dropdown and Slider) placed side by side with padding
            html.Div([
                html.Div([
                    html.Label('Dropdown'),
                    dcc.Dropdown(
                        id='country-dropdown',
                        options=[{'label': i, 'value': i} for i in np.sort(data_tidy['country'].unique())],
                        value='United States',
                        multi=True
                    ),
                ], style={'display': 'inline-block', 'width': 'calc(50% - 10px)', 'padding-right': '5px'}),  # Adjust width calculation here

                html.Div([
                    html.Label('Slider'),
                    dcc.RangeSlider(
                        id='year-range-slider',
                        min=int(data_tidy['year'].min()),
                        max=int(data_tidy['year'].max()),
                        step=1,
                        value=[int(data_tidy['year'].min()), int(data_tidy['year'].max())],
                        marks=None,
                        tooltip={"placement": "bottom", "always_visible": True}
                    ),
                ], style={'display': 'inline-block', 'width': 'calc(50% - 10px)', 'padding-left': '5px'}),  # And here
            ], style={'width': '100%', 'margin-bottom': '20px'}),

            dcc.Graph(id='line-chart', style={'height': '600px'}),
            html.Div('Text box for Graph', style={'textAlign': 'center'}),
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding-right': '10px'}),

        # Polar bear image and its associated text box
        html.Div([
            html.Div([], style={'height': '20px'}),  # This empty div acts as a spacer
            html.Img(src=app.get_asset_url('polarbear.png'), style={'width': '100%', 'height': 'auto'}),
            html.Div('Polar Bear Text Box', style={'textAlign': 'center'}),
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding-left': '10px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

], style={'height': '100vh', 'padding': '10px'})


@app.callback(
    Output('line-chart', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('year-range-slider', 'value')]
)
def update_graph(selectedCountry, selectedYears):
    if not isinstance(selectedCountry, list) or not selectedCountry:
        # If selectedCountry is not a list (e.g., it's 'Select' or None) or it's empty, initialize it with 'United States'.
        selectedCountry = ['United States']

    # Convert years to strings because the DataFrame has years as strings.
    selectedYears = [str(y) for y in selectedYears]

    # Filter the data based on selected years.
    filtered_data = data_tidy[(data_tidy['year'] >= selectedYears[0]) & (data_tidy['year'] <= selectedYears[1])]

    # If there are countries selected, further filter the data.
    if selectedCountry:
        filtered_data = filtered_data[filtered_data['country'].isin(selectedCountry)]

    # Now, create the figure with the filtered data.
    fig = px.line(filtered_data, x='year', y='temperature_change', color='country',
                  title='Climate Change per Country and Year')

    fig.update_layout(
        xaxis_title='Years',
        yaxis_title='Temperature Change',
        legend_title='Countries',
        xaxis=dict(tickmode='linear', tick0=0, dtick=10, tickangle=-45),
        yaxis=dict(tickmode='auto', nticks=10),
    )
    return fig

if __name__ == '__main__':
    app.run(jupyter_mode='tab', debug=True)


    

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>